In [1]:
# Load libraries
import os
import json
import random
import requests
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import xml.etree.ElementTree as ET
from pyproj import Proj, transform
from typing import List, Tuple, Dict, Optional

# Load ENV
load_dotenv('.env', verbose=True)

# Set seed
random.seed(42)
np.random.seed(42)

# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# set path
main_path = os.getcwd()
dataset_path = os.path.join(main_path, 'data')
trans_dataset_path = os.path.join(main_path, 'trans_data')

In [2]:
file_list = os.listdir(dataset_path)
file_list

['서울시 자치구별 연료별 차종별 용도별 등록현황(24년03월).xls',
 '서울특별시_주유소 현황_20220101.csv',
 '어린이집기본정보조회(정기)-기준일(20240331).xls',
 '서울시 소방서,안전센터,구조대 위치정보.csv',
 '한국가스안전공사_수소충전소 현황_20220828.csv',
 '전국초중등학교위치표준데이터.csv',
 '인구밀도_20240407160650.csv',
 '한국가스안전공사_전국 LPG 충전소 현황_20220901.csv',
 '서울시 경로당 정보.csv']

In [3]:
# Kakao map api
def get_location(address: str) -> Optional[List[str]]:
    map_api = f'KakaoAK {os.getenv("KAKAO_MAP_REST_API_KEY")}'
    try:
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
        headers = {"Authorization": map_api}
        api_json = json.loads(str(requests.get(url, headers=headers).text))
        if api_json == None or len(api_json) == 0:
            raise Exception('API response is empty')
        res = api_json['documents']
        if len(res) == 0:
            raise Exception(f'No result found with address: {address}')
        return [res[0]['y'], res[0]['x']]
    except Exception as e:
        # print(e)
        return None


def get_location_str(address=str) -> Optional[str]:
    location = get_location(address)
    if location == None:
        return None
    return ','.join(location)


get_location_str('서울 광진구 중곡동 611-7')

'37.5600856475688,127.073722068721'

In [4]:
"""   ITFR2000_MTM 좌표계를 WGS84 좌표계로 변환   """


# ITRF2000_MTM 좌표계 정의
proj_itrf2000_mtm = Proj(
    "+proj=tmerc +lat_0=38N +lon_0=127E +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs")

# WGS84 좌표계 정의
proj_wgs84 = Proj(proj='latlong', datum='WGS84')


def get_wgs84(x: float, y: float, round_digit: int = 8) -> Tuple[float, float]:
    # 좌표 변환 실행
    long, lat = transform(proj_itrf2000_mtm, proj_wgs84, x, y)
    long, lat = long + 0.002, lat + 0.898  # 좌표 보정
    return round(lat, round_digit), round(long, round_digit)


def get_wgs84_str(x: float, y: float, round_digit: int = 8) -> str:
    lat, long = get_wgs84(x, y, round_digit)
    return f'{lat},{long}'


# 좌표 변환 실행
lon, lat = get_wgs84(203801.8368, 545809.8871)
print("Longitude:", lon, "Latitude:", lat)

Longitude: 38.40975717 Latitude: 127.04500157


### LPG 충전소 위치

서울에 존재하는 LPG 충전소의 위치 정보를 가져온다.


In [5]:
# 원본 데이터 불러오기
df = pd.read_csv(
    os.path.join(dataset_path, '한국가스안전공사_전국 LPG 충전소 현황_20220901.csv'), encoding="cp949")

# 행정구역이 서울인 데이터만 추출
df = df[df['행정구역'].str.split(' ', expand=True)[0] == '서울']

# 소재지를 이용하여 위도, 경도 추출
df = (df.join(df['소재지'].apply(get_location_str).dropna()
              .str.split(',', expand=True)
              .rename(columns={0: 'lat', 1: 'long'}))
        .reset_index(drop=True))

# CSV 저장
df.to_csv(os.path.join(trans_dataset_path, 'lpg_station.csv'),
          index=False, encoding='utf-8')
df.tail(3)

,행정구역,업소명,소재지,전화번호,관리구분,lat,long
69,서울 중랑구,망우충전소,서울특별시 중랑구 망우로 449,024325400,자동차충전,37.6006676333535,127.101862137537
70,서울 중랑구,동일석유(주)황금충전소,서울특별시 중랑구 망우로 209,024352627,자동차충전,37.5941757814607,127.07594887553
71,서울 중랑구,신내LPG충전소,서울특별시 중랑구 용마산로 691,0222085151,자동차충전,37.6128164609389,127.100946121544


### 주유소 위치

서울에 존재하는 주유소의 위치 정보를 가져온다.


In [7]:
# 원본 데이터 불러오기
df = pd.read_csv(
    os.path.join(dataset_path, '서울특별시_주유소 현황_20220101.csv'), encoding="cp949")

# 주소를 이용하여 위도, 경도 추출
df = (df.join(df['주소'].apply(get_location_str).dropna()
              .str.split(',', expand=True)
              .rename(columns={0: 'lat', 1: 'long'}))
        .reset_index(drop=True))

# CSV 저장
df.to_csv(os.path.join(trans_dataset_path, 'gas_station.csv'),
          index=False, encoding='utf-8')
df.tail(3)

,연번,자치구명,주유소명,주소,lat,long
470,471,도봉구,(주)송만에너지 도봉제일주유소,서울특별시 도봉구 도봉로 783 (도봉동),37.6744735831616,127.044066665278
471,472,도봉구,노원교주유소,서울특별시 도봉구 마들로 776 (도봉동),37.679014779402,127.049750654086
472,473,도봉구,오복주유소,서울특별시 도봉구 방학로 43 (방학동),37.6622801187124,127.047441496273


### 수소 충전소 위치

서울에 존재하며 연구용이 아닌 수소 충전소의 위치 정보를 가져온다.


In [8]:
# 원본 데이터 불러오기
df = pd.read_csv(
    os.path.join(dataset_path, '한국가스안전공사_수소충전소 현황_20220828.csv'), encoding="cp949")

# 행정구역이 서울인 데이터만 추출
df = df[df['주소'].str.contains('서울')]

# 수소 충전소의 용도가 연구가 아닌 경우만 추춘
df = df[df['용도'] != '연구용']

# 주소를 이용하여 위도, 경도 추출
df = (df.join(df['주소'].apply(get_location_str).dropna()
              .str.split(',', expand=True)
              .rename(columns={0: 'lat', 1: 'long'}))
        .reset_index(drop=True))

# CSV 저장
df.to_csv(os.path.join(trans_dataset_path, 'hydrogen_station.csv'),
          index=False, encoding='utf-8')
df.head(3)

,번호,충전소명,구분,공급방식,주소,용도,lat,long
0,3,서울특별시 양재그린카스테이션,저장식,튜브트레일러,서울 서초구 바우뫼로12길 73,상업용,37.4683779486418,127.034373688113
1,4,서울특별시 상암수소스테이션,제조식,추출(NG+매립가스),서울 마포구 하늘공원로 86,상업용,37.5715979723206,126.881674368525
2,27,수소에너지네트워크(주) 국회 수소충전소,저장식,튜브트레일러,서울 영등포구 의사당대로 1,상업용,37.5329119742768,126.916403869603


### 초중교 학교 위치

수소충전소는 폭팔 위험성이 있음으로 대피 취약 인구가 밀집되어 있는 학교에서 가까울수록 수소충전소 입지 적합도가 하락한다.

또한 법적으로 전국 초중고 학교에서 최소 7m 떨어져 있어야 한다.

이에 서울에 존재하는 운영중인 초중교 학교의 위치 정보를 가져온다.


In [9]:
# 원본 데이터 불러오기
df = pd.read_csv(
    os.path.join(dataset_path, '전국초중등학교위치표준데이터.csv'), encoding="cp949")

# 행정구역이 서울인 데이터만 추출
df = df[df['소재지도로명주소'].str.split(' ', expand=True)[0].str.contains('서울')]

# 운영상태가 운영중인 학교만 추출
df = df[df['운영상태'] == '운영']

# 위도와 경도가 없는 데이터는 제외
df = df.dropna(subset=['위도', '경도'])

# 이름 변경(통일화)
df = df.rename(columns={'위도': 'lat', '경도': 'long'})

# CSV 저장
df.to_csv(os.path.join(trans_dataset_path, 'school_location.csv'),
          index=False, encoding='utf-8')
df.tail(1)

,학교ID,학교명,학교급구분,설립일자,설립형태,본교분교구분,운영상태,소재지지번주소,소재지도로명주소,시도교육청코드,시도교육청명,교육지원청코드,교육지원청명,생성일자,변경일자,lat,long,데이터기준일자,제공기관코드,제공기관명
11989,B000002000,서울대동초등학교,초등학교,1980-10-02,공립,본교,운영,서울특별시 영등포구 대림동 702,서울특별시 영등포구 대림로21길 6,7010000,서울특별시교육청,7041000,서울특별시남부교육지원청,2013-11-29,2023-11-15,37.493705,126.900119,2024-03-22,C738100,청주대학교 지방교육재정연구원


### 어린이집 위치

수소충전소는 폭팔 위험성이 있음으로 대피 취약 인구가 밀집되어 있는 어린이집에서 가까울수록 수소충전소 입지 적합도가 하락한다.

또한 법적으로 전국 어린이집에서 최소 7m 떨어져 있어야 한다.

이에 서울에 존재하는 운영 중인 어린이집의 위치 정보를 가져온다.


In [15]:
# 원본 데이터 불러오기
df = pd.read_excel(
    os.path.join(dataset_path, '어린이집기본정보조회(정기)-기준일(20240331).xls'))

# 행정구역이 서울인 데이터만 추출
df = df[df['시도'].str.contains('서울')]

# 운영상태가 운영중인 학교만 추출
df = df[df['운영현황'] == '정상']

# 위도와 경도가 없는 데이터는 제외
df = df.dropna(subset=['위도', '경도'])

# 이름 변경(통일화)
df = df.rename(columns={'위도': 'lat', '경도': 'long'})

# CSV 저장
df.to_csv(os.path.join(trans_dataset_path, 'baby_school_location.csv'),
          index=False, encoding='utf-8')
df.tail(1)

,시도,시군구,어린이집명,어린이집유형구분,운영현황,우편번호,주소,어린이집전화번호,어린이집팩스번호,보육실수,...,정원수,현원수,lat,long,통학차량운영여부,홈페이지주소,인가일자,휴지시작일자,휴지종료일자,폐지일자
9341,서울특별시,강동구,그린키즈어린이집,가정,정상,5206.0,"서울특별시 강동구 아리수로 375 104동 121호(고덕동, 고덕풍경채어바니티)",02-426-1555,--,4,...,20,5,37.564003,127.168169,미운영,NaN,2024-03-13,NaN,NaN,NaN


### 경로당 위치

수소충전소가 폭팔 할 경우 거동이 불편하신 어르신들은 대피하기 어렵다.

경로당에 가까울수록 적합한 수소충전소 부지로 선정하기 어렵다.

이에 서울에 존재하는 운영 중인 경로당의 위치 정보를 가져온다.


In [16]:
# 원본 데이터 불러오기
df = pd.read_csv(
    os.path.join(dataset_path, '서울시 경로당 정보.csv'), encoding="cp949")

# 행정구역이 서울인 데이터만 추출
df = (df[df['소재지전체주소'].str.split(' ', expand=True)[0]
         .str.contains('서울')].dropna(subset=['소재지전체주소']))

# 영업상태명이 운영중인 학교만 추출
df = df[df['영업상태명'] == '운영중']

# 주소를 이용하여 위도, 경도 추출
df = df.join(df['소재지전체주소'].apply(get_location_str).dropna()
             .str.split(',', expand=True)
             .rename(columns={0: 'lat', 1: 'long'}))

# lat 또는 long이 NaN인 데이터는 제외
df = df.dropna(subset=['lat', 'long']).reset_index(drop=True)

# CSV 저장
df.to_csv(os.path.join(trans_dataset_path, 'elderly_center.csv'),
          index=False, encoding='utf-8')
df.tail(1)

,번호,사업장명,소재지전체주소,도로명전체주소,인허가일자,영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지우편번호,입소정원,자격소유인원수,총인원수,인허가번호,상세영업상태명,lat,long
3216,2,백련산힐스테이트1차아파트경로당,서울특별시 은평구 응암동,NaN,20131121,운영중,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3110000201500015,운영,37.5963272964127,126.924109745075


### 서울시 지점별 교통량

수소충전소는 교통량이 많은 지역에 설치되어야 한다. 이에 따라 Open API를 통해 지점 정보와 실시간 교통량 정보를 가져와 저장한다.


In [11]:
# 교통량 측정 지점 정보 저장
def get_traffic_location() -> pd.DataFrame:
    df = pd.DataFrame(
        columns=["spot_num", "spot_nm", "grs80tm_x", "grs80tm_y"])
    api_key = os.getenv('TRAFFIC_LOCATION_API_KEY')
    # max page: 139
    url = f"http://openapi.seoul.go.kr:8088/{api_key}/xml/SpotInfo/1/139/"
    response = requests.get(url)
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        for row in root.findall('.//row'):
            data = {
                'spot_num': row.find('spot_num').text,
                'spot_nm': row.find('spot_nm').text,
                'grs80tm_x': row.find('grs80tm_x').text,
                'grs80tm_y': row.find('grs80tm_y').text
            }
            df.loc[len(df)] = data

    df = df.join(df.apply(lambda ser: get_wgs84_str(ser['grs80tm_x'], ser['grs80tm_y']), axis=1).dropna()
                 .str.split(',', expand=True)
                 .rename(columns={0: 'lat', 1: 'long'}))
    return df


# 지점별 교통량 총합 가져오기(동일한 기간의 동일한 시점에서)
def get_traffic_volume_list(
        traffic_location: pd.DataFrame,
        YMD_list=['20240312', '20240313', '20240314',
                  '20240315', '20240316', '20240317', '20240318'],
        HH_list=['00', '06', '12', '18']) -> List[int]:
    location_list = traffic_location.loc[:, 'spot_num'].tolist()
    api_key = os.getenv('TRAFFIC_RECORD_API_KEY')
    traffic_volume_list = []

    for location in tqdm(location_list):
        vol_sum = 0
        for YMD in YMD_list:
            for HH in HH_list:
                url = f"http://openapi.seoul.go.kr:8088/{api_key}/"
                url += f"xml/VolInfo/1/100/{location}/{YMD}/{HH}/"
                try:
                    response = requests.get(url)
                    if response.status_code == 200:
                        root = ET.fromstring(response.content)
                        for row in root.findall('.//row'):
                            vol_sum += int(row.find('vol').text)
                except Exception as e:
                    print(e)
                    vol_sum += 0
        traffic_volume_list.append(vol_sum)
    return traffic_volume_list


traffic_location = get_traffic_location()
traffic_location['traffic_volume'] = get_traffic_volume_list(traffic_location)

traffic_location.to_csv(os.path.join(
    trans_dataset_path, 'traffic.csv'), index=False, encoding='utf-8')
traffic_location.tail(3)

100%|█████████████████████████████████████████████████████████████████████████████| 139/139 [41:27<00:00, 17.89s/it]


,spot_num,spot_nm,grs80tm_x,grs80tm_y,lat,long,traffic_volume
136,F-08,강남순환로(관악터널),191832,437667,37.4352777,126.9107865,105101
137,F-09,서부간선지하도로,189706,441044,37.46568897,126.88700004,60619
138,F-10,신월여의지하도로,187350,447719,37.52581182,126.86057019,67316


### 자치구별 수소차 등록 대수

수소차가 많이 등록된 자치구일수록 수소충전소 사용 비율이 높을 것으로 예상된다.

파일은 복합 엑셀 문서로 수작업으로 직접 수소차 등록 대수를 추출하였다.


In [12]:
districts = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중량구', '성북구', '강북구', '도봉구', '노원구', '은평구',
             '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']
counts = [79, 58, 93, 69, 67, 65, 59, 73, 41, 47, 84, 134, 108,
          186, 126, 265, 200, 64, 177, 128, 78, 236, 269, 241, 273,]

# save as csv
df = pd.DataFrame({'district': districts, 'count': counts})
df.to_csv(os.path.join(trans_dataset_path, 'hydrogen_car_count.csv'),
          index=False, encoding='utf-8')
df.T

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
district,종로구,중구,용산구,성동구,광진구,동대문구,중량구,성북구,강북구,도봉구,...,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구
count,79,58,93,69,67,65,59,73,41,47,...,265,200,64,177,128,78,236,269,241,273


### 서울 소방서 위치 정보

폭팔 및 화제 위험시 소방서가 가까울수록 대응하기 좋다.


In [21]:
# 원본 데이터 불러오기
df = pd.read_csv(
    os.path.join(dataset_path, "서울시 소방서,안전센터,구조대 위치정보.csv"), encoding="cp949")

df = df.join(df.apply(lambda ser: get_wgs84_str(ser['X좌표'], ser['Y좌표']), axis=1).dropna()
             .str.split(',', expand=True)
             .rename(columns={0: 'lat', 1: 'long'}))

# 소방서만 추출
fire_station = df[df['유형구분명'] == '소방서'].reset_index(drop=True)

# CSV 저장
fire_station.to_csv(os.path.join(trans_dataset_path, 'fire_station.csv'),
                    index=False, encoding='utf-8')
fire_station.tail(3)

,연번,서ㆍ센터ID,서ㆍ센터명,유형구분명,도서지역포함여부,상위서ㆍ센터ID,일련번호,X좌표,Y좌표,lat,long
21,161,1118000,강동소방서,소방서,,1100000,26,211043.487669,547760.417187,38.42727325,127.12693915
22,175,1111000,노원소방서,소방서,,1100000,40,206278.710106,559835.894278,38.53611713,127.07313692
23,176,1110000,도봉소방서,소방서,,1100000,41,203797.404500,562711.422384,38.56203874,127.04503903


In [22]:
df.columns

Index(['연번', '서ㆍ센터ID', '서ㆍ센터명', '유형구분명', '도서지역포함여부', '상위서ㆍ센터ID', '일련번호', 'X좌표',
       'Y좌표', 'lat', 'long'],
      dtype='object')

### 서울 구조대 및 안전센터 위치 정보

폭팔 및 화제 위험시 구조대 및 안전센터가 가까울수록 대응하기 좋다.

하지만 소방서와 달리 실질적으로 화제에 대응하는 역할을 하는 곳이 아니기 때문에 수소충전소 입지 적합도에는 큰 영향을 미치지 않을 것으로 예상기에 소방서와 구분하여 데이터를 정리한다.


In [23]:
# 소방서가 아닌 안전센터/구조대 추출
rescue_station = df[df['유형구분명'] != '소방서'].reset_index(drop=True)

# CSV 저장
rescue_station.to_csv(os.path.join(trans_dataset_path, 'rescue_station.csv'),
                      index=False, encoding='utf-8')
rescue_station.tail(3)

,연번,서ㆍ센터ID,서ㆍ센터명,유형구분명,도서지역포함여부,상위서ㆍ센터ID,일련번호,X좌표,Y좌표,lat,long
150,173,1125102,우이119안전센터,안전센터/구조대,,1125000,38,201461.941319,560141.801092,38.53889359,127.01856421
151,174,1125103,미아119안전센터,안전센터/구조대,,1125000,39,202500.723258,557825.979950,38.51802612,127.03032599
152,177,1111102,상계119안전센터,안전센터/구조대,,1111000,42,206323.823000,561714.886221,38.55304622,127.07366431


### 서울 동별 인구밀도

인구밀도가 적당히 높을수록 수소충전소에 대한 수요가 많을 것으로 예상된다.

하지만 지나치게 높으면 인명피해 위험성이 크며 지나치게 작으면 이용률이 저조할 것이다.

이 기준값은 머신러닝 모델이 찾을 것이다. 이를 위해 서울 동별 인구밀도를 조사한다.


In [24]:
# 원본 데이터 불러오기
df = pd.read_csv(
    os.path.join(dataset_path, '인구밀도_20240407160650.csv'), encoding="utf-8")

# Refactoring
df.columns = df.iloc[0, :]
df = df.drop(0)
df['주소'] = df.apply(lambda df: f"{df['동별(2)']} {df['동별(3)']}", axis=1)
df = (df[df['주소'].str.contains('소계') == False]
      .drop(columns=['동별(1)', '동별(2)', '동별(3)'])
      .reset_index(drop=True))

# CSV 저장
df.to_csv(os.path.join(trans_dataset_path, 'population_density.csv'),
          index=False, encoding='utf-8')
df.tail(3)

,인구 (명),면적 (㎢),인구밀도 (명/㎢),주소
423,33057,2.26,14627,강동구 강일동
424,39178,1.82,21526,강동구 상일1동
425,11933,1.09,10948,강동구 상일2동
